# Client Example

In [1]:
from langserve import RemoteRunnable

In [2]:
from typing import Optional, List
from pydantic import BaseModel, Field

class Person(BaseModel):
    age: Optional[int] = Field(None, description="The age of the person in years.")
    name: Optional[str] = Field(None, description="The name of the person.")
    nick_name: Optional[str] = Field(None, description="Alias, if any.")

In [3]:
runnable = RemoteRunnable("http://localhost:8000/extract_text/")

In [4]:
text = """
My name is Chester. i am 42 years old. My friend Jane is a year older than me.
"""

In [5]:
response = runnable.invoke({"text": text, "schema": Person.schema()})
response

{'data': [{'name': 'Chester', 'age': 42}, {'name': 'Jane'}]}

Add instructions:

In [6]:
instructions = "Redact all names using the characters `######`"

response = runnable.invoke(
    {
        "text": text,
        "schema": Person.schema(),
        "instructions": instructions,
    }
)
response

{'data': [{'name': 'Chester', 'age': 42}, {'name': 'Jane'}]}

Add few-shot examples:

In [7]:
instructions = "Redact all names using the characters `######`"
examples = [
    {
        "text": "My name is Grung. I am 100.",
        "output": [ {"age": 100, "name": "######", "hello": "meow"}] ,
    }
]

response = runnable.invoke(
    {
        "text": text,
        "schema": Person.schema(),
        "instructions": instructions,
        "examples": examples,
    }
)
response

{'data': [{'name': '######', 'age': 42}, {'name': '######', 'age': 43}]}

## Persist extractors

In [8]:
import requests

In [9]:
url = "http://localhost:8000"

In [10]:
data = {
    "description": "Extract references to people, having properties name and age.",
    "schema": Person.schema(),
    "instruction": "Redact all names using the characters `######`",
}

response = requests.post(f"{url}/extractors", json=data)
response

<Response [200]>

In [11]:
uuid = response.json()
print(uuid)

05f4fed2-c07e-4322-a605-f781846dc493


### Add examples

In [12]:
import json

examples = [
    {
        "text": "My name is Grung. I am 100.",
        "output": [Person(age=100, name="######").dict()],
    }
]

responses = []
for example in examples:
    create_request = {
        "extractor_id": uuid,
        "content": example["text"],
        "output": example['output'],
    }
    response = requests.post(f"{url}/examples", json=create_request)
    responses.append(response)

In [13]:
response = requests.get(f"{url}/examples?extractor_id={uuid}")
response

<Response [200]>

In [14]:
response.json()

[{'output': [{'age': 100, 'name': '######', 'nick_name': None}],
  'id': 'cb0a4b38-491b-4548-9703-89fdbd350876',
  'content': 'My name is Grung. I am 100.',
  'extractor_id': '05f4fed2-c07e-4322-a605-f781846dc493',
  'uuid': 'ef7ae889-7036-4eeb-a0ff-af03827cf0e4',
  'created_at': '2024-02-26T20:11:13.756379',
  'updated_at': '2024-02-26T20:11:13.756382'}]

### Extract using persisted extractor

In [19]:
request_data = {"extractor_id": uuid, "text": text}
print(request_data)

{'extractor_id': '05f4fed2-c07e-4322-a605-f781846dc493', 'text': '\nMy name is Chester. i am 42 years old. My friend Jane is a year older than me.\n'}


In [20]:
response = requests.post(f"{url}/extract", data=request_data)
response

<Response [200]>

In [21]:
response.text

'{"data":[{"age":42,"name":"######","nick_name":null}]}'